In [154]:
import pandas as pd
from collections import defaultdict
import json, math, gdown
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import requests
pd.options.display.float_format = '{:.2f}'.format
import requests

In [206]:

# we use this data from each month
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240305.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240403.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240501.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240602.pkl"
# url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240706.pkl"
url = "https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard/resolve/main/elo_results_20240806.pkl"


response = requests.get(url)

with open("data.pkl", "wb") as file:
    file.write(response.content)

battle_info = pd.read_pickle("data.pkl")

battle_info.keys()

# battle_info["text"]["full"].keys()

dict_keys(['text', 'vision'])

In [207]:
battle_info["text"]["full"]["leaderboard_table_df"].sort_values("rating", ascending=False)
# battle_info["text"]["leaderboard_table_df"].sort_values("rating", ascending=False)
# battle_info["leaderboard_table_df"].sort_values("rating", ascending=False)

,rating,variance,rating_q975,rating_q025,num_battles,final_ranking
gemini-1.5-pro-exp-0801,1299.42,6.31,1303.42,1294.53,15244,1
gpt-4o-2024-05-13,1286.13,2.59,1288.68,1282.30,72589,2
gpt-4o-mini-2024-07-18,1276.81,6.33,1281.08,1271.42,16064,3
claude-3-5-sonnet-20240620,1271.24,3.18,1274.60,1267.63,42939,3
gemini-advanced-0514,1266.38,2.36,1269.36,1263.76,52126,4
...,...,...,...,...,...,...
chatglm-6b,878.85,25.51,886.34,868.44,4998,122
fastchat-t5-3b,868.24,27.25,878.12,858.73,4304,123
stablelm-tuned-alpha-7b,839.55,40.80,852.90,827.41,3333,125
dolly-v2-12b,822.15,30.87,832.61,812.79,3486,125


In [321]:
## LMSYS

# Load the file
file_path = '/Users/buzz/Documents/Haim/Data-lmsys-csv.csv'
df = pd.read_csv(file_path)

# Drop any completely empty columns
df = df.dropna(axis=1, how='all')

# Get the unique models and sorted dates (months) in ascending order
models = df['model'].unique()
months = sorted(df['date'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%y'))

# Normalization data
normalization_data_lmsys = {
    "Mar-24": {"min": 797.51, "max": 1251.07},
    "Apr-24": {"min": 801.24, "max": 1255.91},
    "May-24": {"min": 804.36, "max": 1258.82},
    "Jun-24": {"min": 798.35, "max": 1256.16},
    "Jul-24": {"min": 798.60, "max": 1287.40},
    "Aug-24": {"min": 798.47, "max": 1299.42}
}

# Normalization function
def normalize(value, min_val, max_val):
    return round((value - min_val) / (max_val - min_val), 2)

# Create pivot table
pivot_df = df.pivot(index='model', columns='date', values='rating')

# Normalize the values
for month in months:
    month_str = pd.to_datetime(month, format='%b-%y').strftime('%b-%y')
    if month_str in normalization_data_lmsys:
        min_val = normalization_data_lmsys[month_str]["min"]
        max_val = normalization_data_lmsys[month_str]["max"]
        pivot_df[month] = pivot_df[month].apply(lambda x: normalize(x, min_val, max_val) if pd.notnull(x) else x)

# Reset index to make 'model' a column
result_df_lmsys = pivot_df.reset_index()

# Filter to keep only rows with 4 or more non-null values
result_df_lmsys = result_df_lmsys[result_df_lmsys.notnull().sum(axis=1) >4]

# Calculate the variation between the highest and lowest values in each row
def calculate_variation(row):
    numeric_values = row.dropna()[1:]  # Exclude the model column and drop NaN values
    if len(numeric_values) >= 2:
        return round(((numeric_values.max() - numeric_values.min()) / numeric_values.min()) * 100)
    return None

# Apply the variation calculation to each row
result_df_lmsys['var_%'] = result_df_lmsys.apply(calculate_variation, axis=1)

# Sort the DataFrame by variation_percent in descending order
result_df_lmsys = result_df_lmsys.sort_values('var_%', ascending=False)

# Save the result DataFrame to a new CSV file
result_df_lmsys.to_csv('Model_Month_Combined_Filtered_Normalized_Variation_Lmsys.csv', index=False)

# Display the result DataFrame
result_df_lmsys

date,model,Apr-24,Aug-24,Jul-24,Jun-24,Mar-24,May-24,var_%
25,gpt-4-0125-preview,0.99,0.89,0.91,0.98,1.00,0.97,12
7,claude-3-opus-20240229,1.00,0.90,0.92,0.98,NaN,0.98,11
28,gpt-4-1106-preview,0.99,0.90,0.93,0.99,1.00,0.99,11
29,gpt-4-turbo-2024-04-09,NaN,0.91,0.94,1.00,NaN,1.00,10
14,gemini-1.5-pro-api-0409-preview,NaN,0.92,0.94,1.00,NaN,0.98,9
1,bard-jan-24-gemini-pro,0.89,NaN,0.84,0.90,0.90,0.89,7
8,claude-3-sonnet-20240229,0.88,NaN,0.82,0.88,NaN,0.87,7
26,gpt-4-0314,0.85,NaN,NaN,0.85,0.86,0.85,1


In [320]:
## HELM Lite

# Load the file
file_path = '/Users/buzz/Documents/Haim/Data-helm-lite.csv'
df = pd.read_csv(file_path)

# Drop any completely empty columns
df = df.dropna(axis=1, how='all')

# Get the unique models and sorted dates (months) in ascending order
models = df['model'].unique()
months = sorted(df['date'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%y'))

# Normalization data
normalization_data_helm = {
    "Mar-24": {"min": 0.045, "max": 0.965},
    "Apr-24": {"min": 0.05, "max": 0.958},
    "May-24": {"min": 0.072, "max": 0.957},
    "Jun-24": {"min": 0.064, "max": 0.973},
    "Jul-24": {"min": 0.06, "max": 0.965},
    "Aug-24": {"min": 0.052, "max": 0.963}
}

# Normalization function
def normalize(value, min_val, max_val):
    return round((value - min_val) / (max_val - min_val), 2)

# Create pivot table
pivot_df = df.pivot(index='model', columns='date', values='rating')

# Normalize the values
for month in months:
    month_str = pd.to_datetime(month, format='%b-%y').strftime('%b-%y')
    if month_str in normalization_data_helm:
        min_val = normalization_data_helm[month_str]["min"]
        max_val = normalization_data_helm[month_str]["max"]
        pivot_df[month] = pivot_df[month].apply(lambda x: normalize(x, min_val, max_val) if pd.notnull(x) else x)

# Reset index to make 'model' a column
result_df_helm_lite = pivot_df.reset_index()

# Filter to keep only rows with 4 or more non-null values
result_df_helm_lite = result_df_helm_lite[result_df_helm_lite.notnull().sum(axis=1) >4]

# Calculate the variation between the highest and lowest values in each row
def calculate_variation(row):
    numeric_values = row.dropna()[1:]  # Exclude the model column and drop NaN values
    if len(numeric_values) >= 2:
        return round(((numeric_values.max() - numeric_values.min()) / numeric_values.min()) *100)
    return None

# Apply the variation calculation to each row
result_df_helm_lite['var_%'] = result_df_helm_lite.apply(calculate_variation, axis=1)

# Sort the DataFrame by variation_percent in descending order
result_df_helm_lite = result_df_helm_lite.sort_values('var_%', ascending=False)

# Save the result DataFrame to a new CSV file
result_df_helm_lite.to_csv('Model_Month_Combined_Filtered_Normalized_Variation_Helm_Lite.csv', index=False)


# Display the result DataFrame
result_df_helm_lite

date,model,Apr-24,Aug-24,Jul-24,Jun-24,Mar-24,May-24,var_%
32,PaLM-2 (Bison),0.55,NaN,NaN,0.53,0.55,0.54,4
24,Llama 3 (70B),0.80,0.80,0.79,0.78,NaN,0.80,3
30,Mixtral (8x22B),0.72,0.72,0.71,0.70,NaN,0.71,3
33,PaLM-2 (Unicorn),0.70,0.69,0.69,0.68,0.69,0.69,3
35,Palmyra X V3 (72B),0.70,0.69,0.69,0.68,0.69,0.69,3
37,Qwen1.5 (32B),0.67,NaN,0.66,0.65,NaN,0.66,3
41,Yi (34B),0.67,NaN,0.66,0.65,0.66,0.66,3
31,Mixtral (8x7B 32K seqlen),0.66,NaN,0.65,0.65,0.66,0.66,2
34,Palmyra X V2 (33B),0.60,0.60,0.59,0.59,0.60,0.59,2
6,Claude 3 Opus (20240229),NaN,0.79,0.79,0.78,NaN,0.79,1


In [ ]:
normalization_mmlu = {
    "Apr-24": {"min": 0.458, "max": 0.846},
    "May-24": {"min": 0.295, "max": 0.846},
    "Jun-24": {"min": 0.295, "max": 0.846},
    "Jul-24": {"min": 0.295, "max": 0.865},
}

In [322]:
# Load the file
file_path = '/Users/buzz/Documents/Haim/Data-llmu_inc.csv'
df = pd.read_csv(file_path)

# Drop any completely empty columns
df = df.dropna(axis=1, how='all')

# Get the unique models and sorted dates (months) in ascending order
models = df['model'].unique()
months = sorted(df['date'].unique(), key=lambda x: pd.to_datetime(x, format='%b-%y'))

# Normalization data for MMLU
normalization_data_mmlu = {
    "Apr-24": {"min": 0.458, "max": 0.846},
    "May-24": {"min": 0.295, "max": 0.846},
    "Jun-24": {"min": 0.295, "max": 0.846},
    "Jul-24": {"min": 0.295, "max": 0.865}
}

# Normalization function
def normalize(value, min_val, max_val):
    return round((value - min_val) / (max_val - min_val), 2)

# Create pivot table
pivot_df = df.pivot(index='model', columns='date', values='rating')

# Normalize the values
for month in months:
    month_str = pd.to_datetime(month, format='%b-%y').strftime('%b-%y')
    if month_str in normalization_data_mmlu:
        min_val = normalization_data_mmlu[month_str]["min"]
        max_val = normalization_data_mmlu[month_str]["max"]
        pivot_df[month] = pivot_df[month].apply(lambda x: normalize(x, min_val, max_val) if pd.notnull(x) else x)

# Reset index to make 'model' a column
result_df_mmlu = pivot_df.reset_index()

# Filter to keep only rows with 4 or more non-null values
result_df_mmlu = result_df_mmlu[result_df_mmlu.notnull().sum(axis=1) > 3]

# Calculate the variation between the highest and lowest values in each row
def calculate_variation(row):
    numeric_values = row.dropna()[1:]  # Exclude the model column and drop NaN values
    if len(numeric_values) >= 2:
        return round(((numeric_values.max() - numeric_values.min()) / numeric_values.min()) * 100)
    return None

# Apply the variation calculation to each row
result_df_mmlu['var_%'] = result_df_mmlu.apply(calculate_variation, axis=1)

# Sort the DataFrame by variation_percent in descending order
result_df_mmlu = result_df_mmlu.sort_values('var_%', ascending=False)

# Save the result DataFrame to a new CSV file
result_df_mmlu.to_csv('Model_Month_Combined_Filtered_Normalized_Variation_MMLU.csv', index=False)

# Display the result DataFrame
result_df_mmlu


date,model,Apr-24,Jul-24,Jun-24,May-24,var_%
34,Yi (34B),0.78,0.82,0.85,0.85,9
5,Claude 3 Sonnet (20240229),0.78,0.81,0.84,0.84,8
30,Qwen1.5 (72B),0.81,0.84,0.87,0.87,7
11,GPT-4 Turbo (1106 preview),0.87,0.88,0.91,0.91,5
25,PaLM-2 (Unicorn),0.85,0.86,0.89,0.89,5
22,Mixtral (8x22B),NaN,0.85,0.88,0.88,4
4,Claude 3 Opus (20240229),1.00,0.97,1.00,1.00,3
10,GPT-4 (0613),0.94,0.93,0.96,0.96,3
18,Gemini 1.5 Pro (0409 preview),NaN,0.90,0.93,0.93,3
20,Llama 3 (70B),NaN,0.87,0.90,0.90,3
